# 第7章：批归一化与Dropout

> **学习目标：**
> - 理解Batch Normalization的原理和作用
> - 掌握Dropout的工作机制
> - 理解训练模式和推理模式的差异
> - 学会组合使用多种正则化技术
> - 了解Layer Normalization等变体
>
> **学习时间：** 3-4小时

---

## 📚 为什么需要Batch Normalization和Dropout？

### 深度网络的挑战

1. **内部协变量偏移（Internal Covariate Shift）**
   - 每层输入的分布随训练不断变化
   - 导致训练不稳定，需要小学习率
   - **解决方案**：Batch Normalization

2. **过拟合（Overfitting）**
   - 网络记住训练数据细节
   - 泛化能力差
   - **解决方案**：Dropout

---

## 📖 目录

1. **Batch Normalization（批归一化）**
   - 原理和数学推导
   - 训练时和推理时的不同
   - 从零实现
   - 可训练参数γ和β

2. **Dropout**
   - 工作原理
   - 训练模式 vs 推理模式
   - 从零实现
   - Inverted Dropout

3. **Layer Normalization**
   - BN vs LN对比
   - 适用场景

4. **综合实战**
   - MNIST数据集
   - 对比不同正则化技术组合

---

In [ ]:
# 导入必要的库
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rcParams
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 设置中文字体
rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei']
rcParams['axes.unicode_minus'] = False

# 设置随机种子
np.random.seed(42)

print("环境配置完成！")

In [ ]:
# 提取统计量
iterations = range(n_iterations)
means_without_bn = [stat['mean'] for stat in net_without_bn.layer2_input_stats]
stds_without_bn = [stat['std'] for stat in net_without_bn.layer2_input_stats]
means_with_bn = [stat['mean'] for stat in net_with_bn.layer2_input_stats]
stds_with_bn = [stat['std'] for stat in net_with_bn.layer2_input_stats]

# 创建可视化
fig = plt.figure(figsize=(16, 10))
gs = fig.add_gridspec(3, 2, hspace=0.3, wspace=0.3)

# ========== 第1行：均值和标准差变化 ==========
ax1 = fig.add_subplot(gs[0, 0])
ax1.plot(iterations, means_without_bn, color='#e74c3c', linewidth=2, label='无BN')
ax1.fill_between(iterations, 
                  np.array(means_without_bn) - np.array(stds_without_bn),
                  np.array(means_without_bn) + np.array(stds_without_bn),
                  color='#e74c3c', alpha=0.2)
ax1.set_xlabel('训练迭代次数', fontsize=11)
ax1.set_ylabel('第2层输入值', fontsize=11)
ax1.set_title('❌ 无BN：分布剧烈变化（内部协变量偏移）', fontsize=12, fontweight='bold')
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)

ax2 = fig.add_subplot(gs[0, 1])
ax2.plot(iterations, means_with_bn, color='#2ecc71', linewidth=2, label='有BN')
ax2.fill_between(iterations,
                  np.array(means_with_bn) - np.array(stds_with_bn),
                  np.array(means_with_bn) + np.array(stds_with_bn),
                  color='#2ecc71', alpha=0.2)
ax2.set_xlabel('训练迭代次数', fontsize=11)
ax2.set_ylabel('第2层输入值', fontsize=11)
ax2.set_title('✅ 有BN：分布稳定（均值≈0，方差≈1）', fontsize=12, fontweight='bold')
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)

# ========== 第2行：分布的直方图对比（选择几个时间点）==========
snapshots = [0, 15, 30, 45]  # 选择4个时间点
colors_timeline = ['#3498db', '#9b59b6', '#f39c12', '#e74c3c']

ax3 = fig.add_subplot(gs[1, 0])
for i, snapshot_idx in enumerate(snapshots):
    data = net_without_bn.layer2_input_stats[snapshot_idx]['data'].flatten()
    ax3.hist(data, bins=30, alpha=0.5, label=f'迭代{snapshot_idx}', 
             color=colors_timeline[i], density=True)
ax3.set_xlabel('激活值', fontsize=11)
ax3.set_ylabel('密度', fontsize=11)
ax3.set_title('无BN：分布随训练不断偏移', fontsize=12, fontweight='bold')
ax3.legend(fontsize=9)
ax3.grid(True, alpha=0.3, axis='y')

ax4 = fig.add_subplot(gs[1, 1])
for i, snapshot_idx in enumerate(snapshots):
    data = net_with_bn.layer2_input_stats[snapshot_idx]['data'].flatten()
    ax4.hist(data, bins=30, alpha=0.5, label=f'迭代{snapshot_idx}',
             color=colors_timeline[i], density=True)
ax4.set_xlabel('激活值', fontsize=11)
ax4.set_ylabel('密度', fontsize=11)
ax4.set_title('有BN：分布始终保持标准化', fontsize=12, fontweight='bold')
ax4.legend(fontsize=9)
ax4.grid(True, alpha=0.3, axis='y')

# ========== 第3行：统计量变化曲线 ==========
ax5 = fig.add_subplot(gs[2, 0])
ax5.plot(iterations, means_without_bn, color='#e74c3c', linewidth=2, label='均值')
ax5.plot(iterations, stds_without_bn, color='#3498db', linewidth=2, label='标准差')
ax5.set_xlabel('训练迭代次数', fontsize=11)
ax5.set_ylabel('统计量数值', fontsize=11)
ax5.set_title('无BN：均值和方差都在变化', fontsize=12, fontweight='bold')
ax5.legend(fontsize=10)
ax5.grid(True, alpha=0.3)

ax6 = fig.add_subplot(gs[2, 1])
ax6.plot(iterations, means_with_bn, color='#e74c3c', linewidth=2, label='均值')
ax6.plot(iterations, stds_with_bn, color='#3498db', linewidth=2, label='标准差')
ax6.axhline(y=0, color='gray', linestyle='--', linewidth=1, alpha=0.5, label='目标均值=0')
ax6.axhline(y=1, color='gray', linestyle='--', linewidth=1, alpha=0.5, label='目标标准差=1')
ax6.set_xlabel('训练迭代次数', fontsize=11)
ax6.set_ylabel('统计量数值', fontsize=11)
ax6.set_title('有BN：均值≈0，标准差≈1（稳定）', fontsize=12, fontweight='bold')
ax6.legend(fontsize=9)
ax6.grid(True, alpha=0.3)

plt.suptitle('内部协变量偏移（Internal Covariate Shift）可视化', 
             fontsize=14, fontweight='bold', y=0.995)
plt.show()

print("\n" + "="*70)
print("🔍 观察要点：")
print("="*70)
print("\n【问题】无BN的网络：")
print("  1. 第2层输入的均值和方差随训练不断变化（上图左侧）")
print("  2. 分布在不同迭代中偏移很大（中图左侧，曲线分散）")
print("  3. 统计量不稳定，难以选择合适的学习率（下图左侧）")
print("\n【解决方案】有BN的网络：")
print("  4. 第2层输入始终保持均值≈0，方差≈1（上图右侧）")
print("  5. 分布在不同迭代中重叠良好（中图右侧，曲线集中）")
print("  6. 统计量稳定，可以使用更大的学习率（下图右侧）")
print("\n💡 这就是为什么需要Batch Normalization！")
print("="*70)

In [ ]:
# 模拟内部协变量偏移的演示
np.random.seed(42)

# 生成简单的分类数据
n_samples = 500
X_demo = np.random.randn(n_samples, 10)
y_demo = (X_demo[:, 0] + X_demo[:, 1] > 0).astype(int)

# 简单的2层网络（用于演示）
class SimpleNetwork:
    """简单网络，用于演示内部协变量偏移"""
    def __init__(self, input_size=10, hidden_size=20, use_bn=False):
        self.W1 = np.random.randn(hidden_size, input_size) * 0.5
        self.b1 = np.zeros(hidden_size)
        self.W2 = np.random.randn(1, hidden_size) * 0.5
        self.b2 = np.zeros(1)
        self.use_bn = use_bn
        
        # 记录第2层输入的统计量
        self.layer2_input_stats = []
        
        if use_bn:
            self.bn = BatchNormalization(hidden_size)
    
    def forward(self, X, record_stats=True):
        # 第1层
        z1 = X @ self.W1.T + self.b1
        a1 = relu(z1)
        
        # 如果使用BN
        if self.use_bn:
            a1 = self.bn.forward(a1, training=True)
        
        # 记录第2层的输入统计量
        if record_stats:
            self.layer2_input_stats.append({
                'mean': np.mean(a1),
                'std': np.std(a1),
                'data': a1.copy()  # 保存一份数据用于可视化
            })
        
        # 第2层
        z2 = a1 @ self.W2.T + self.b2
        return z2
    
    def train_step(self, X, y, lr=0.01):
        """简化的训练步骤（只更新W1，观察分布变化）"""
        # 前向传播
        output = self.forward(X, record_stats=True)
        
        # 简化的梯度下降（只更新第一层权重）
        # 这里不做完整的反向传播，只随机扰动权重来模拟训练
        self.W1 += np.random.randn(*self.W1.shape) * lr

# 训练两个网络：有BN vs 无BN
print("训练两个网络（50个迭代）...")
print("  - 网络1：无BN")
print("  - 网络2：有BN\n")

net_without_bn = SimpleNetwork(use_bn=False)
net_with_bn = SimpleNetwork(use_bn=True)

n_iterations = 50
for i in range(n_iterations):
    # 随机采样一个batch
    batch_indices = np.random.choice(n_samples, 64, replace=False)
    X_batch = X_demo[batch_indices]
    y_batch = y_demo[batch_indices]
    
    # 训练步骤
    net_without_bn.train_step(X_batch, y_batch)
    net_with_bn.train_step(X_batch, y_batch)

print("✅ 训练完成！现在来看看第2层输入分布的变化...")

---

## 🎬 可视化演示：内部协变量偏移问题

在学习Batch Normalization之前，让我们先直观地看看**内部协变量偏移**是什么，以及它为什么是个问题。

---

## 1. Batch Normalization（批归一化）

### 1.1 原理和数学推导

**核心思想：** 将每一层的输入标准化到均值0、方差1。

**算法流程（训练时）：**

给定一个mini-batch的输入 $\mathbf{X} = \{x_1, x_2, ..., x_m\}$：

1. **计算mini-batch均值：**
   $$\mu_B = \frac{1}{m} \sum_{i=1}^{m} x_i$$

2. **计算mini-batch方差：**
   $$\sigma_B^2 = \frac{1}{m} \sum_{i=1}^{m} (x_i - \mu_B)^2$$

3. **标准化：**
   $$\hat{x}_i = \frac{x_i - \mu_B}{\sqrt{\sigma_B^2 + \epsilon}}$$
   
   其中 $\epsilon$ 是防止除零的小常数（如1e-5）

4. **缩放和平移（可学习参数）：**
   $$y_i = \gamma \hat{x}_i + \beta$$
   
   - $\gamma$：缩放参数（scale）
   - $\beta$：平移参数（shift）
   - 这两个参数是**可学习的**，通过反向传播更新

**为什么需要γ和β？**

标准化会限制表示能力。通过γ和β，网络可以学习到：
- 如果 $\gamma = \sqrt{\sigma_B^2}$, $\beta = \mu_B$，则恢复原始输入
- 否则，学习到最优的分布

### 1.2 从零实现Batch Normalization

In [ ]:
# 可视化计算图和梯度流
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# ========== 左图：前向传播计算图 ==========
ax = axes[0]
ax.set_xlim(0, 10)
ax.set_ylim(0, 10)
ax.axis('off')
ax.set_title('前向传播计算图', fontsize=14, fontweight='bold', pad=20)

# 节点位置
nodes = {
    'X': (1, 5),
    'mu': (3, 7),
    'var': (3, 3),
    'X-mu': (5, 5),
    'X_hat': (7, 5),
    'Y': (9, 5)
}

# 绘制节点
for name, (x, y) in nodes.items():
    if name in ['X', 'Y']:
        color = '#3498db'
        size = 800
    else:
        color = '#95a5a6'
        size = 600
    ax.scatter(x, y, s=size, color=color, alpha=0.8, zorder=3)
    ax.text(x, y, name, ha='center', va='center', fontsize=11, 
            fontweight='bold', color='white', zorder=4)

# 绘制边（带箭头）
edges = [
    ('X', 'mu', '1/m·Σ'),
    ('X', 'var', '1/m·Σ(·-μ)²'),
    ('X', 'X-mu', ''),
    ('mu', 'X-mu', ''),
    ('X-mu', 'X_hat', '÷√(σ²+ε)'),
    ('var', 'X_hat', ''),
    ('X_hat', 'Y', 'γ·+β')
]

for start, end, label in edges:
    x1, y1 = nodes[start]
    x2, y2 = nodes[end]
    ax.annotate('', xy=(x2, y2), xytext=(x1, y1),
                arrowprops=dict(arrowstyle='->', lw=2, color='#2c3e50'))
    # 标签
    if label:
        mid_x, mid_y = (x1 + x2) / 2, (y1 + y2) / 2
        ax.text(mid_x, mid_y + 0.3, label, fontsize=9, ha='center',
                bbox=dict(boxstyle='round,pad=0.3', facecolor='white', 
                         edgecolor='none', alpha=0.8))

ax.text(5, 9, '前向传播：X → 均值/方差 → 标准化 → 缩放平移 → Y', 
        ha='center', fontsize=10, style='italic', color='#7f8c8d')

# ========== 右图：反向传播路径 ==========
ax = axes[1]
ax.set_xlim(0, 10)
ax.set_ylim(0, 10)
ax.axis('off')
ax.set_title('反向传播梯度流（3条路径）', fontsize=14, fontweight='bold', pad=20)

# 节点
for name, (x, y) in nodes.items():
    if name in ['X', 'Y']:
        color = '#e74c3c'
        size = 800
    else:
        color = '#95a5a6'
        size = 600
    ax.scatter(x, y, s=size, color=color, alpha=0.8, zorder=3)
    ax.text(x, y, name, ha='center', va='center', fontsize=11,
            fontweight='bold', color='white', zorder=4)

# 反向传播路径（颜色区分）
backward_edges = [
    # 路径1：直接路径（红色）
    (('Y', 'X_hat', '∂L/∂Y·γ', '#e74c3c'), ('X_hat', 'X-mu', '÷std', '#e74c3c')),
    # 路径2：通过方差（蓝色）
    (('X_hat', 'var', '∂²σ/∂x̂', '#3498db'), ('var', 'X-mu', '', '#3498db')),
    # 路径3：通过均值（绿色）
    (('X_hat', 'mu', '∂μ/∂x̂', '#2ecc71'), ('mu', 'X', '', '#2ecc71')),
]

# 路径1
ax.annotate('', xy=nodes['X_hat'], xytext=nodes['Y'],
            arrowprops=dict(arrowstyle='->', lw=3, color='#e74c3c'))
ax.annotate('', xy=nodes['X-mu'], xytext=nodes['X_hat'],
            arrowprops=dict(arrowstyle='->', lw=3, color='#e74c3c'))
ax.annotate('', xy=nodes['X'], xytext=nodes['X-mu'],
            arrowprops=dict(arrowstyle='->', lw=3, color='#e74c3c'))

# 路径2
ax.annotate('', xy=nodes['var'], xytext=nodes['X_hat'],
            arrowprops=dict(arrowstyle='->', lw=2.5, color='#3498db', 
                          connectionstyle="arc3,rad=0.3"))
ax.annotate('', xy=nodes['X'], xytext=nodes['var'],
            arrowprops=dict(arrowstyle='->', lw=2.5, color='#3498db',
                          connectionstyle="arc3,rad=-0.3"))

# 路径3  
ax.annotate('', xy=nodes['mu'], xytext=nodes['X_hat'],
            arrowprops=dict(arrowstyle='->', lw=2.5, color='#2ecc71',
                          connectionstyle="arc3,rad=-0.3"))
ax.annotate('', xy=nodes['X'], xytext=nodes['mu'],
            arrowprops=dict(arrowstyle='->', lw=2.5, color='#2ecc71',
                          connectionstyle="arc3,rad=0.3"))

# 图例
legend_y = 1
ax.text(0.5, legend_y, '━━', color='#e74c3c', fontsize=12, fontweight='bold')
ax.text(1.2, legend_y, '路径1: 直接路径', fontsize=9)
ax.text(0.5, legend_y-0.7, '━━', color='#3498db', fontsize=12, fontweight='bold')
ax.text(1.2, legend_y-0.7, '路径2: 通过σ²', fontsize=9)
ax.text(0.5, legend_y-1.4, '━━', color='#2ecc71', fontsize=12, fontweight='bold')
ax.text(1.2, legend_y-1.4, '路径3: 通过μ', fontsize=9)

ax.text(5, 9, '反向传播：dY → dX (需合并3条路径的梯度)', 
        ha='center', fontsize=10, style='italic', color='#7f8c8d')

plt.tight_layout()
plt.show()

print("\n💡 从计算图理解反向传播：")
print("  - 前向传播：输入X分别计算μ和σ²，然后标准化")
print("  - 反向传播：梯度从Y回流到X有3条路径，需要全部累加")
print("  - 这就是为什么BN的backward代码看起来复杂的原因！")

In [ ]:
print("="*70)
print("📐 Batch Normalization 反向传播 - 分步推导")
print("="*70)

# 生成测试数据
np.random.seed(42)
m = 4  # batch size（使用小数据便于手算验证）
n = 3  # 特征数
X_test = np.array([
    [1.0, 2.0, 3.0],
    [4.0, 5.0, 6.0],
    [7.0, 8.0, 9.0],
    [10.0, 11.0, 12.0]
])

print(f"\n📥 输入数据 X (shape={X_test.shape}):")
print(X_test)

# 前向传播（手动计算每一步）
print("\n" + "-"*70)
print("🔽 前向传播（保存中间变量）")
print("-"*70)

# Step 1: 计算均值
mu = np.mean(X_test, axis=0)
print(f"\n【Step 1】计算batch均值 μ = (1/m) Σx_i")
print(f"  μ = {mu}")
print(f"  公式: μ_j = (x₁ⱼ + x₂ⱼ + ... + x_mⱼ) / m")

# Step 2: 计算方差
var = np.var(X_test, axis=0)
print(f"\n【Step 2】计算batch方差 σ² = (1/m) Σ(x_i - μ)²")
print(f"  σ² = {var}")
print(f"  公式: σ²_j = Σ(x_ij - μ_j)² / m")

# Step 3: 计算 x - mu
x_minus_mu = X_test - mu
print(f"\n【Step 3】计算偏差 (x - μ)")
print(x_minus_mu)

# Step 4: 计算标准差
epsilon = 1e-5
std = np.sqrt(var + epsilon)
print(f"\n【Step 4】计算标准差 std = √(σ² + ε)")
print(f"  std = {std}")
print(f"  ε = {epsilon}（防止除零）")

# Step 5: 标准化
X_hat = x_minus_mu / std
print(f"\n【Step 5】标准化 x̂ = (x - μ) / std")
print(X_hat)
print(f"  验证: 均值 = {np.mean(X_hat, axis=0)}（应该≈0）")
print(f"        方差 = {np.var(X_hat, axis=0)}（应该≈1）")

# Step 6: 缩放和平移
gamma = np.ones(n)  # 假设γ=1
beta = np.zeros(n)  # 假设β=0
Y = gamma * X_hat + beta
print(f"\n【Step 6】缩放平移 y = γ·x̂ + β")
print(f"  γ = {gamma}")
print(f"  β = {beta}")
print(f"  y = {Y}")

print("\n" + "-"*70)
print("🔼 反向传播（计算梯度）")
print("-"*70)

# 假设上游梯度（从损失函数传回）
dY = np.ones_like(Y)  # 简化：假设损失对y的梯度全为1
print(f"\n📥 上游梯度 dL/dy (shape={dY.shape}):")
print(dY)

# ========== 反向传播计算 ==========

# Step 1: 对γ和β的梯度（直接求导）
print(f"\n【Backward Step 1】计算 ∂L/∂γ 和 ∂L/∂β")
print(f"  y = γ·x̂ + β")
print(f"  → ∂y/∂γ = x̂")
print(f"  → ∂y/∂β = 1")
print(f"  链式法则:")
print(f"    ∂L/∂γ = Σ(∂L/∂y · ∂y/∂γ) = Σ(dY · x̂)")
print(f"    ∂L/∂β = Σ(∂L/∂y · ∂y/∂β) = Σ(dY)")

dgamma = np.sum(dY * X_hat, axis=0)
dbeta = np.sum(dY, axis=0)
print(f"\n  结果:")
print(f"    dγ = {dgamma}")
print(f"    dβ = {dbeta}")

# Step 2: 对x̂的梯度
print(f"\n【Backward Step 2】计算 ∂L/∂x̂")
print(f"  y = γ·x̂ + β → ∂y/∂x̂ = γ")
print(f"  ∂L/∂x̂ = ∂L/∂y · γ")

dX_hat = dY * gamma
print(f"  dX̂ = {dX_hat}")

# Step 3: 对方差的梯度（较复杂）
print(f"\n【Backward Step 3】计算 ∂L/∂σ²")
print(f"  x̂ = (x - μ) / √(σ² + ε)")
print(f"  → ∂x̂/∂σ² = (x - μ) · (-1/2) · (σ² + ε)^(-3/2)")
print(f"  ∂L/∂σ² = Σ(∂L/∂x̂ · ∂x̂/∂σ²)")

dvar = np.sum(dX_hat * x_minus_mu * (-0.5) * (var + epsilon)**(-1.5), axis=0)
print(f"  dσ² = {dvar}")

# Step 4: 对均值的梯度（最复杂，有两条路径）
print(f"\n【Backward Step 4】计算 ∂L/∂μ")
print(f"  注意：μ影响损失的两条路径:")
print(f"    路径1: μ → x̂ → y → L")
print(f"    路径2: μ → σ² → x̂ → y → L")
print(f"  ")
print(f"  路径1梯度: Σ(∂L/∂x̂ · (-1/std))")
print(f"  路径2梯度: ∂L/∂σ² · Σ(-2(x-μ)/m)")

dmu_path1 = np.sum(dX_hat * (-1/std), axis=0)
dmu_path2 = dvar * np.sum(-2 * x_minus_mu, axis=0) / m
dmu = dmu_path1 + dmu_path2
print(f"  路径1: {dmu_path1}")
print(f"  路径2: {dmu_path2}")
print(f"  总梯度: dμ = {dmu}")

# Step 5: 对输入X的梯度（三条路径汇总）
print(f"\n【Backward Step 5】计算 ∂L/∂x（最终梯度）")
print(f"  x影响损失的三条路径:")
print(f"    路径1: x → x̂ → y → L")
print(f"    路径2: x → σ² → x̂ → y → L")  
print(f"    路径3: x → μ → ... → L")
print(f"  ")
print(f"  ∂L/∂x = ∂L/∂x̂·(1/std) + ∂L/∂σ²·(2(x-μ)/m) + ∂L/∂μ·(1/m)")

dX_path1 = dX_hat / std
dX_path2 = dvar * 2 * x_minus_mu / m
dX_path3 = dmu / m
dX = dX_path1 + dX_path2 + dX_path3

print(f"\n  路径1（直接通过x̂）:")
print(dX_path1)
print(f"\n  路径2（通过σ²）:")
print(dX_path2)
print(f"\n  路径3（通过μ）:")
print(dX_path3)
print(f"\n  🎯 最终梯度 dX = dX_path1 + dX_path2 + dX_path3:")
print(dX)

print("\n" + "="*70)
print("✅ 反向传播推导完成！")
print("="*70)
print("\n💡 关键要点:")
print("  1. BN反向传播复杂在于μ和σ²的依赖关系")
print("  2. 每个输入x影响损失有3条路径（直接、通过σ²、通过μ）")
print("  3. 需要小心处理batch维度的求和")
print("  4. 实际代码中这些计算被合并优化，但原理相同")
print("="*70)

### 1.4 Batch Normalization 反向传播详解

**⚠️ 为什么BN的反向传播比较复杂？**

因为每个输入 $x_i$ 的标准化值 $\hat{x}_i$ 不仅依赖于自己，还依赖于整个batch的均值和方差。这导致梯度计算需要考虑这些依赖关系。

**📊 计算图可视化：**

```
输入 X → 均值μ ┐
         方差σ² ┼→ 标准化 X̂ → 缩放平移 Y → 损失L
         X ────┘
```

**🔍 反向传播推导（步骤拆解）：**

我们已知上游梯度 $\frac{\partial L}{\partial y}$，需要计算：
- $\frac{\partial L}{\partial \gamma}$（对缩放参数的梯度）
- $\frac{\partial L}{\partial \beta}$（对平移参数的梯度）
- $\frac{\partial L}{\partial x}$（对输入的梯度，传递给前一层）

让我们一步步推导...

In [ ]:
class BatchNormalization:
    """
    批归一化层
    
    参数:
        n_features: 输入特征数量
        momentum: 用于更新running mean/var的动量（默认0.9）
        epsilon: 防止除零的小常数（默认1e-5）
    """
    def __init__(self, n_features, momentum=0.9, epsilon=1e-5):
        self.n_features = n_features
        self.momentum = momentum
        self.epsilon = epsilon
        
        # 可学习参数（通过反向传播更新）
        self.gamma = np.ones(n_features)   # 缩放参数，初始化为1
        self.beta = np.zeros(n_features)   # 平移参数，初始化为0
        
        # 推理时使用的统计量（训练时累积）
        self.running_mean = np.zeros(n_features)
        self.running_var = np.ones(n_features)
        
        # 缓存的中间变量（用于反向传播）
        self.cache = None
    
    def forward(self, X, training=True):
        """
        前向传播
        
        参数:
            X: 输入数据, shape (m, n_features)
            training: 是否为训练模式
        
        返回:
            out: 归一化后的输出, shape (m, n_features)
        """
        if training:
            # ===== 训练模式 =====
            # 1. 计算mini-batch的均值和方差
            batch_mean = np.mean(X, axis=0)  # shape: (n_features,)
            batch_var = np.var(X, axis=0)    # shape: (n_features,)
            
            # 2. 标准化
            X_normalized = (X - batch_mean) / np.sqrt(batch_var + self.epsilon)
            
            # 3. 缩放和平移
            out = self.gamma * X_normalized + self.beta
            
            # 4. 更新running统计量（用于推理）
            # 指数移动平均: new = momentum * old + (1 - momentum) * current
            self.running_mean = self.momentum * self.running_mean + \
                                (1 - self.momentum) * batch_mean
            self.running_var = self.momentum * self.running_var + \
                               (1 - self.momentum) * batch_var
            
            # 5. 缓存中间变量（反向传播用）
            self.cache = {
                'X': X,
                'X_normalized': X_normalized,
                'batch_mean': batch_mean,
                'batch_var': batch_var,
                'm': X.shape[0]
            }
        else:
            # ===== 推理模式 =====
            # 使用训练时累积的running统计量
            X_normalized = (X - self.running_mean) / \
                          np.sqrt(self.running_var + self.epsilon)
            out = self.gamma * X_normalized + self.beta
        
        return out
    
    def backward(self, dout):
        """
        反向传播
        
        参数:
            dout: 上游梯度, shape (m, n_features)
        
        返回:
            dX: 对输入的梯度, shape (m, n_features)
        
        同时更新 self.grad_gamma 和 self.grad_beta
        """
        # 从cache中取出前向传播的中间变量
        X = self.cache['X']
        X_normalized = self.cache['X_normalized']
        batch_mean = self.cache['batch_mean']
        batch_var = self.cache['batch_var']
        m = self.cache['m']
        
        # 1. 对γ和β的梯度
        self.grad_gamma = np.sum(dout * X_normalized, axis=0)  # shape: (n_features,)
        self.grad_beta = np.sum(dout, axis=0)                  # shape: (n_features,)
        
        # 2. 对标准化后的X的梯度
        dX_normalized = dout * self.gamma  # shape: (m, n_features)
        
        # 3. 对方差的梯度
        # ∂L/∂σ² = Σ ∂L/∂x̂ * (x - μ) * (-0.5) * (σ² + ε)^(-3/2)
        dvar = np.sum(dX_normalized * (X - batch_mean) * \
                      (-0.5) * (batch_var + self.epsilon) ** (-3/2), axis=0)
        
        # 4. 对均值的梯度
        # ∂L/∂μ = Σ ∂L/∂x̂ * (-1/√(σ²+ε)) + ∂L/∂σ² * Σ(-2(x-μ)/m)
        dmean = np.sum(dX_normalized * (-1 / np.sqrt(batch_var + self.epsilon)), axis=0) + \
                dvar * np.sum(-2 * (X - batch_mean), axis=0) / m
        
        # 5. 对输入X的梯度
        dX = dX_normalized / np.sqrt(batch_var + self.epsilon) + \
             dvar * 2 * (X - batch_mean) / m + \
             dmean / m
        
        return dX


print("BatchNormalization实现完成！")
print("\n💡 关键点：")
print("1. 训练时：使用batch统计量，更新running统计量")
print("2. 推理时：使用running统计量（固定）")
print("3. γ和β是可学习参数，通过反向传播更新")

In [ ]:
# 可视化对比结果
fig = plt.figure(figsize=(18, 10))
gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

epochs = np.arange(1, 151)
colors = {
    '❌ 无正则化': '#95a5a6',
    '🔵 只有BN': '#3498db',
    '🟢 只有Dropout': '#2ecc71',
    '⭐ BN+Dropout': '#e74c3c'
}

# ========== 第1行：训练和测试准确率 ==========
ax1 = fig.add_subplot(gs[0, :2])
for config_name, result in results.items():
    ax1.plot(epochs, result['train_accs'], '--', 
             color=colors[config_name], alpha=0.4, linewidth=1.5)
    ax1.plot(epochs, result['test_accs'], '-', 
             color=colors[config_name], label=config_name, linewidth=2.5)
ax1.set_xlabel('Epoch', fontsize=11)
ax1.set_ylabel('准确率', fontsize=11)
ax1.set_title('学习曲线对比（实线=测试，虚线=训练）', fontsize=12, fontweight='bold')
ax1.legend(loc='lower right', fontsize=10)
ax1.grid(True, alpha=0.3)
ax1.set_ylim(0, 1.05)

# ========== 第1行右侧：最终性能对比 ==========
ax2 = fig.add_subplot(gs[0, 2])
config_names = list(results.keys())
final_test_accs = [results[name]['final_test_acc'] for name in config_names]
bars = ax2.barh(range(len(config_names)), final_test_accs, 
                color=[colors[name] for name in config_names], alpha=0.8)
ax2.set_yticks(range(len(config_names)))
ax2.set_yticklabels(config_names, fontsize=9)
ax2.set_xlabel('最终测试准确率', fontsize=10)
ax2.set_title('性能排名', fontsize=12, fontweight='bold')
ax2.grid(True, alpha=0.3, axis='x')
for i, bar in enumerate(bars):
    width = bar.get_width()
    ax2.text(width, bar.get_y() + bar.get_height()/2, 
             f'{width:.3f}', ha='left', va='center', fontsize=9, 
             fontweight='bold')

# ========== 第2行：损失曲线 ==========
ax3 = fig.add_subplot(gs[1, :2])
for config_name, result in results.items():
    ax3.plot(epochs, result['train_losses'], '--',
             color=colors[config_name], alpha=0.4, linewidth=1.5)
    ax3.plot(epochs, result['test_losses'], '-',
             color=colors[config_name], label=config_name, linewidth=2.5)
ax3.set_xlabel('Epoch', fontsize=11)
ax3.set_ylabel('损失值', fontsize=11)
ax3.set_title('损失曲线对比（实线=测试，虚线=训练）', fontsize=12, fontweight='bold')
ax3.legend(loc='upper right', fontsize=10)
ax3.grid(True, alpha=0.3)
ax3.set_ylim(0, 3)

# ========== 第2行右侧：过拟合分析 ==========
ax4 = fig.add_subplot(gs[1, 2])
overfitting_gaps = [results[name]['overfitting_gap'] for name in config_names]
bars = ax4.barh(range(len(config_names)), overfitting_gaps,
                color=[colors[name] for name in config_names], alpha=0.8)
ax4.set_yticks(range(len(config_names)))
ax4.set_yticklabels(config_names, fontsize=9)
ax4.set_xlabel('过拟合程度\n(训练准确率 - 测试准确率)', fontsize=10)
ax4.set_title('过拟合分析', fontsize=12, fontweight='bold')
ax4.grid(True, alpha=0.3, axis='x')
ax4.axvline(x=0.05, color='red', linestyle='--', linewidth=1, alpha=0.5)
ax4.text(0.05, -0.5, '警戒线', fontsize=8, color='red')
for i, bar in enumerate(bars):
    width = bar.get_width()
    ax4.text(width, bar.get_y() + bar.get_height()/2,
             f'{width:.3f}', ha='left', va='center', fontsize=9,
             fontweight='bold')

# ========== 第3行：收敛速度对比 ==========
ax5 = fig.add_subplot(gs[2, :])
# 计算每个配置达到90%准确率所需的epoch数
convergence_epochs = {}
for config_name, result in results.items():
    test_accs = np.array(result['test_accs'])
    idx = np.where(test_accs >= 0.9)[0]
    if len(idx) > 0:
        convergence_epochs[config_name] = idx[0] + 1
    else:
        convergence_epochs[config_name] = 150  # 未达到

# 绘制收敛速度
x_offset = 0
for config_name in config_names:
    epochs_to_converge = convergence_epochs[config_name]
    result = results[config_name]
    
    # 绘制达到90%准确率的过程
    if epochs_to_converge < 150:
        ax5.plot(range(1, epochs_to_converge+1), 
                result['test_accs'][:epochs_to_converge],
                color=colors[config_name], linewidth=3, 
                label=f"{config_name} ({epochs_to_converge} epochs)")
        ax5.scatter(epochs_to_converge, 0.9, 
                   color=colors[config_name], s=150, zorder=5,
                   marker='*', edgecolors='black', linewidths=1.5)
    else:
        ax5.plot(epochs[:100], result['test_accs'][:100],
                color=colors[config_name], linewidth=3, alpha=0.5,
                linestyle='--', label=f"{config_name} (未达到)")

ax5.axhline(y=0.9, color='gray', linestyle='--', linewidth=1.5, alpha=0.7)
ax5.text(5, 0.91, '目标: 90%准确率', fontsize=10, color='gray')
ax5.set_xlabel('Epoch', fontsize=11)
ax5.set_ylabel('测试准确率', fontsize=11)
ax5.set_title('收敛速度对比（达到90%准确率所需时间）', fontsize=12, fontweight='bold')
ax5.legend(loc='lower right', fontsize=10)
ax5.grid(True, alpha=0.3)
ax5.set_xlim(0, 100)
ax5.set_ylim(0, 1)

plt.suptitle('BN vs Dropout 全面对比实验', fontsize=15, fontweight='bold', y=0.995)
plt.show()

# 打印详细分析
print("\n" + "="*80)
print("📊 实验结果分析")
print("="*80)

print("\n【1. 收敛速度】（达到90%测试准确率所需epoch数）")
for config_name in config_names:
    epochs_needed = convergence_epochs[config_name]
    if epochs_needed < 150:
        print(f"  {config_name}: {epochs_needed} epochs")
    else:
        print(f"  {config_name}: >150 epochs (未达到)")

print("\n【2. 最终性能】（150 epoch后的测试准确率）")
sorted_results = sorted(results.items(), 
                       key=lambda x: x[1]['final_test_acc'], 
                       reverse=True)
for i, (config_name, result) in enumerate(sorted_results, 1):
    print(f"  {i}. {config_name}: {result['final_test_acc']:.4f}")

print("\n【3. 过拟合程度】（训练准确率 - 测试准确率）")
sorted_overfitting = sorted(results.items(),
                           key=lambda x: x[1]['overfitting_gap'])
for i, (config_name, result) in enumerate(sorted_overfitting, 1):
    gap = result['overfitting_gap']
    status = "✓ 良好" if gap < 0.05 else "⚠️ 轻微过拟合" if gap < 0.1 else "❌ 严重过拟合"
    print(f"  {i}. {config_name}: {gap:.4f} {status}")

print("\n" + "="*80)
print("💡 核心结论")
print("="*80)
print("\n1. 🚀 收敛速度：")
print("   - BN（有无Dropout）明显加快收敛")
print("   - 原因：稳定的梯度流，允许更大的学习率")

print("\n2. 🎯 防止过拟合：")
print("   - Dropout显著降低过拟合")
print("   - BN也有轻微的正则化效果")

print("\n3. ⭐ 最佳实践：")
print("   - BN+Dropout组合：既快速收敛，又防止过拟合")
print("   - 适用场景：深度网络、大型数据集")

print("\n4. 📋 选择建议：")
print("   - 浅层网络：可能不需要BN和Dropout")
print("   - 中等网络：选择BN或Dropout之一")
print("   - 深度网络：建议BN+Dropout组合")
print("="*80)

In [ ]:
# 准备实验数据
print("🔧 准备实验数据...")
digits = load_digits()
X = digits.data / 16.0  # 归一化到[0,1]
y = digits.target

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

# One-hot编码
def to_one_hot(y, n_classes=10):
    one_hot = np.zeros((len(y), n_classes))
    one_hot[np.arange(len(y)), y] = 1
    return one_hot

y_train_oh = to_one_hot(y_train)
y_test_oh = to_one_hot(y_test)

print(f"✅ 数据准备完成")
print(f"   训练集: {X_train.shape}, 测试集: {X_test.shape}")


# 创建简化的训练函数（模拟训练过程）
def simulate_training(use_bn, use_dropout, n_epochs=150, learning_rate=0.01):
    """
    模拟训练过程，记录损失和准确率
    
    注：这里为了演示效果，使用简化的训练模拟
    """
    np.random.seed(42)
    
    # 初始化网络
    input_size, hidden_size, output_size = 64, 128, 10
    
    # 权重
    W1 = np.random.randn(hidden_size, input_size) * 0.01
    b1 = np.zeros(hidden_size)
    W2 = np.random.randn(output_size, hidden_size) * 0.01
    b2 = np.zeros(output_size)
    
    # BN/Dropout层
    bn1 = BatchNormalization(hidden_size) if use_bn else None
    dropout1 = Dropout(0.3) if use_dropout else None
    
    # 记录指标
    train_losses, test_losses = [], []
    train_accs, test_accs = [], []
    
    # 训练循环
    for epoch in range(n_epochs):
        # === 训练阶段 ===
        # 前向传播
        z1 = X_train @ W1.T + b1
        if bn1:
            z1 = bn1.forward(z1, training=True)
        a1 = relu(z1)
        if dropout1:
            a1 = dropout1.forward(a1, training=True)
        
        z2 = a1 @ W2.T + b2
        a2 = softmax(z2)
        
        # 计算损失
        epsilon = 1e-15
        a2_clipped = np.clip(a2, epsilon, 1 - epsilon)
        train_loss = -np.mean(np.sum(y_train_oh * np.log(a2_clipped), axis=1))
        train_acc = np.mean(np.argmax(a2, axis=1) == y_train)
        
        # === 测试阶段 ===
        z1_test = X_test @ W1.T + b1
        if bn1:
            z1_test = bn1.forward(z1_test, training=False)
        a1_test = relu(z1_test)
        if dropout1:
            a1_test = dropout1.forward(a1_test, training=False)
        
        z2_test = a1_test @ W2.T + b2
        a2_test = softmax(z2_test)
        
        a2_test_clipped = np.clip(a2_test, epsilon, 1 - epsilon)
        test_loss = -np.mean(np.sum(y_test_oh * np.log(a2_test_clipped), axis=1))
        test_acc = np.mean(np.argmax(a2_test, axis=1) == y_test)
        
        # 记录
        train_losses.append(train_loss)
        test_losses.append(test_loss)
        train_accs.append(train_acc)
        test_accs.append(test_acc)
        
        # === 简化的梯度下降（用于演示）===
        # 计算输出层梯度
        dz2 = (a2 - y_train_oh) / len(y_train_oh)
        dW2 = dz2.T @ a1
        db2 = np.sum(dz2, axis=0)
        
        # 计算隐藏层梯度
        da1 = dz2 @ W2
        if dropout1:
            da1 = dropout1.backward(da1)
        dz1 = da1 * (z1 > 0)  # ReLU导数
        if bn1:
            dz1 = bn1.backward(dz1)
        dW1 = dz1.T @ X_train
        db1 = np.sum(dz1, axis=0)
        
        # 更新参数
        lr = learning_rate
        # BN允许使用更大的学习率
        if use_bn:
            lr *= 3.0
        
        W2 -= lr * dW2
        b2 -= lr * db2
        W1 -= lr * dW1
        b1 -= lr * db1
        
        # 更新BN参数
        if bn1:
            bn1.gamma -= lr * bn1.grad_gamma
            bn1.beta -= lr * bn1.grad_beta
    
    return {
        'train_losses': train_losses,
        'test_losses': test_losses,
        'train_accs': train_accs,
        'test_accs': test_accs,
        'final_train_acc': train_accs[-1],
        'final_test_acc': test_accs[-1],
        'overfitting_gap': train_accs[-1] - test_accs[-1]
    }


print("\n🚀 开始对比实验...")
print("   这将需要一些时间，请耐心等待...\n")

# 运行4种配置
configs = [
    {'use_bn': False, 'use_dropout': False, 'name': '❌ 无正则化'},
    {'use_bn': True,  'use_dropout': False, 'name': '🔵 只有BN'},
    {'use_bn': False, 'use_dropout': True,  'name': '🟢 只有Dropout'},
    {'use_bn': True,  'use_dropout': True,  'name': '⭐ BN+Dropout'},
]

results = {}
for i, config in enumerate(configs, 1):
    print(f"[{i}/4] 训练配置: {config['name']}", end=" ... ")
    result = simulate_training(
        use_bn=config['use_bn'],
        use_dropout=config['use_dropout'],
        n_epochs=150
    )
    results[config['name']] = result
    print(f"✓ (测试准确率: {result['final_test_acc']:.3f})")

print("\n✅ 所有实验完成！")

---

## 🆚 BN vs Dropout 详细对比实验

现在我们已经理解了BN和Dropout的原理，让我们通过实验来直观对比它们的效果：

**实验设计：**
- 4种配置：无正则化、只用BN、只用Dropout、BN+Dropout
- 对比维度：训练速度、收敛性、过拟合程度
- 数据集：MNIST手写数字（sklearn.datasets）

### 1.3 可视化Batch Normalization的效果

In [ ]:
# 生成测试数据
np.random.seed(42)
X_test = np.random.randn(100, 3) * 5 + 10  # 均值10，标准差5

print("原始数据统计：")
print(f"均值: {np.mean(X_test, axis=0)}")
print(f"方差: {np.var(X_test, axis=0)}")

# 应用Batch Normalization
bn = BatchNormalization(n_features=3)
X_normalized = bn.forward(X_test, training=True)

print("\nBatch Normalization后：")
print(f"均值: {np.mean(X_normalized, axis=0)}")
print(f"方差: {np.var(X_normalized, axis=0)}")

# 可视化
plt.figure(figsize=(12, 4))

# 原始数据分布
plt.subplot(1, 2, 1)
for i in range(3):
    plt.hist(X_test[:, i], bins=20, alpha=0.5, label=f'特征{i+1}')
plt.xlabel('数值', fontsize=11)
plt.ylabel('频数', fontsize=11)
plt.title('原始数据分布（不同尺度）', fontsize=12, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)

# BN后的数据分布
plt.subplot(1, 2, 2)
for i in range(3):
    plt.hist(X_normalized[:, i], bins=20, alpha=0.5, label=f'特征{i+1}')
plt.xlabel('数值', fontsize=11)
plt.ylabel('频数', fontsize=11)
plt.title('Batch Normalization后（均值0，方差1）', fontsize=12, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n🔍 观察要点：")
print("1. BN后所有特征都标准化到相同尺度")
print("2. 有利于梯度传播和训练稳定性")
print("3. 可以使用更大的学习率")

---

## 2. Dropout

### 2.1 Dropout原理

**核心思想：** 训练时随机"关闭"一部分神经元，防止过拟合。

**算法流程：**

**训练时：**
1. 以概率 $p$ 随机将神经元输出设为0
2. 将剩余神经元的输出乘以 $\frac{1}{1-p}$（Inverted Dropout）

**推理时：**
- 不使用Dropout，所有神经元都参与计算

**为什么有效？**
1. **集成学习效果**：相当于训练多个子网络，推理时取平均
2. **减少神经元共适应**：强迫网络学习鲁棒特征
3. **正则化**：限制模型复杂度

### 2.2 从零实现Dropout

In [ ]:
class Dropout:
    """
    Dropout层
    
    参数:
        drop_prob: dropout概率（丢弃神经元的概率）
    
    注意:
        使用Inverted Dropout，训练时缩放，推理时不变
    """
    def __init__(self, drop_prob=0.5):
        """
        参数:
            drop_prob: 丢弃概率 p（0到1之间）
                      p=0.5表示丢弃50%的神经元
        """
        assert 0 <= drop_prob < 1, "drop_prob应该在[0, 1)之间"
        self.drop_prob = drop_prob
        self.mask = None
    
    def forward(self, X, training=True):
        """
        前向传播
        
        参数:
            X: 输入数据, shape (m, n_features)
            training: 是否为训练模式
        
        返回:
            out: shape (m, n_features)
        """
        if training:
            # ===== 训练模式（Inverted Dropout）=====
            # 1. 生成mask：以概率(1-p)保留神经元
            #    mask[i] = 1 表示保留，0 表示丢弃
            self.mask = (np.random.rand(*X.shape) > self.drop_prob).astype(float)
            
            # 2. 应用mask并缩放
            # 缩放因子: 1/(1-p)，使得期望值保持不变
            # 例如：p=0.5时，保留的神经元乘以2
            out = X * self.mask / (1 - self.drop_prob)
        else:
            # ===== 推理模式 =====
            # 不做任何操作，直接返回
            out = X
        
        return out
    
    def backward(self, dout):
        """
        反向传播
        
        参数:
            dout: 上游梯度, shape (m, n_features)
        
        返回:
            dX: 对输入的梯度, shape (m, n_features)
        """
        # 梯度只通过保留的神经元传播
        dX = dout * self.mask / (1 - self.drop_prob)
        return dX


print("Dropout实现完成！")
print("\n💡 Inverted Dropout的优势：")
print("1. 训练时缩放，推理时直接使用（无需额外操作）")
print("2. 推理速度更快")
print("3. 代码更简洁")

### 2.3 可视化Dropout的效果

In [ ]:
# 测试Dropout
np.random.seed(42)
X_dropout = np.ones((5, 10))  # 5个样本，10个特征，全为1

dropout = Dropout(drop_prob=0.5)

# 训练模式
X_train_out = dropout.forward(X_dropout, training=True)
mask = dropout.mask

# 推理模式
X_test_out = dropout.forward(X_dropout, training=False)

# 可视化
fig, axes = plt.subplots(1, 3, figsize=(15, 3))

# 原始输入
axes[0].imshow(X_dropout, cmap='RdYlGn', aspect='auto', vmin=0, vmax=2)
axes[0].set_title('原始输入（全为1）', fontsize=12, fontweight='bold')
axes[0].set_xlabel('特征维度')
axes[0].set_ylabel('样本')
for i in range(5):
    for j in range(10):
        axes[0].text(j, i, f'{X_dropout[i, j]:.1f}', 
                     ha='center', va='center', fontsize=8)

# 训练模式（Dropout激活）
axes[1].imshow(X_train_out, cmap='RdYlGn', aspect='auto', vmin=0, vmax=2)
axes[1].set_title('训练模式（p=0.5，保留的乘以2）', fontsize=12, fontweight='bold')
axes[1].set_xlabel('特征维度')
axes[1].set_ylabel('样本')
for i in range(5):
    for j in range(10):
        axes[1].text(j, i, f'{X_train_out[i, j]:.1f}', 
                     ha='center', va='center', fontsize=8,
                     color='red' if X_train_out[i, j] == 0 else 'black')

# 推理模式（无Dropout）
axes[2].imshow(X_test_out, cmap='RdYlGn', aspect='auto', vmin=0, vmax=2)
axes[2].set_title('推理模式（无Dropout）', fontsize=12, fontweight='bold')
axes[2].set_xlabel('特征维度')
axes[2].set_ylabel('样本')
for i in range(5):
    for j in range(10):
        axes[2].text(j, i, f'{X_test_out[i, j]:.1f}', 
                     ha='center', va='center', fontsize=8)

plt.tight_layout()
plt.show()

print(f"\n统计信息：")
print(f"训练模式 - 保留的神经元比例: {np.mean(mask):.2f} (期望0.5)")
print(f"训练模式 - 输出均值: {np.mean(X_train_out):.2f} (期望1.0)")
print(f"推理模式 - 输出均值: {np.mean(X_test_out):.2f} (期望1.0)")
print("\n💡 Inverted Dropout确保训练和推理时期望值一致！")

In [ ]:
# 可视化决策流程图
fig, axes = plt.subplots(1, 2, figsize=(18, 8))

# ========== 左图：技术选择决策树 ==========
ax = axes[0]
ax.set_xlim(0, 10)
ax.set_ylim(0, 12)
ax.axis('off')
ax.set_title('决策树：选择合适的归一化/正则化技术', fontsize=14, fontweight='bold', pad=20)

# 决策节点
nodes = [
    # (x, y, text, color, size)
    (5, 11, '开始', '#3498db', 1200),
    (3, 9.5, 'CNN?', '#95a5a6', 800),
    (7, 9.5, 'RNN/\nTransformer?', '#95a5a6', 800),
    (2, 8, '✓ BN\n(卷积层)', '#2ecc71', 900),
    (4, 8, '✓ BN+Dropout\n(全连接层)', '#2ecc71', 900),
    (7, 8, '✓ Layer Norm', '#2ecc71', 900),
    (2, 6.5, 'Batch\nSize?', '#95a5a6', 700),
    (4, 6.5, '数据量?', '#95a5a6', 700),
    (7, 6.5, '训练\n稳定?', '#95a5a6', 700),
    (1, 5, '>32:\nBN', '#f39c12', 600),
    (2.5, 5, '<8:\nLayer Norm', '#f39c12', 600),
    (4, 5, '小数据:\n强Dropout', '#f39c12', 600),
    (7, 5, '不稳定:\n加BN', '#f39c12', 600),
]

for x, y, text, color, size in nodes:
    ax.scatter(x, y, s=size, color=color, alpha=0.85, zorder=3)
    ax.text(x, y, text, ha='center', va='center', fontsize=9,
            fontweight='bold', color='white', zorder=4)

# 连接线
edges = [
    (5, 11, 3, 9.5, 'Yes'),
    (5, 11, 7, 9.5, 'No'),
    (3, 9.5, 2, 8, '卷积'),
    (3, 9.5, 4, 8, '全连接'),
    (7, 9.5, 7, 8, 'Yes'),
    (2, 8, 2, 6.5, ''),
    (4, 8, 4, 6.5, ''),
    (7, 8, 7, 6.5, ''),
    (2, 6.5, 1, 5, ''),
    (2, 6.5, 2.5, 5, ''),
    (4, 6.5, 4, 5, ''),
    (7, 6.5, 7, 5, ''),
]

for x1, y1, x2, y2, label in edges:
    ax.annotate('', xy=(x2, y2), xytext=(x1, y1),
                arrowprops=dict(arrowstyle='->', lw=1.5, color='#34495e'))
    if label:
        mid_x, mid_y = (x1 + x2) / 2, (y1 + y2) / 2
        ax.text(mid_x + 0.3, mid_y, label, fontsize=8, style='italic', color='#7f8c8d')

# 图例
ax.text(0.5, 2, '🔵 起点', fontsize=10)
ax.text(0.5, 1.5, '⚪ 判断', fontsize=10)
ax.text(0.5, 1, '🟢 推荐', fontsize=10)
ax.text(0.5, 0.5, '🟡 备选', fontsize=10)

# ========== 右图：使用场景热力图 ==========
ax = axes[1]
ax.set_title('不同场景下的技术选择（颜色越深越推荐）', fontsize=14, fontweight='bold', pad=20)

# 数据
scenarios = [
    'CNN图像分类',
    'CNN目标检测',
    'RNN文本生成',
    'Transformer',
    '小数据集分类',
    '迁移学习',
    '强化学习',
    'GAN生成'
]

techniques = ['BN', 'Layer\nNorm', 'Dropout', 'BN+\nDropout']

# 热力图数据（0-3分，3最推荐）
heatmap_data = np.array([
    [3, 1, 2, 3],  # CNN图像分类
    [2, 2, 2, 3],  # CNN目标检测
    [0, 3, 2, 1],  # RNN文本生成
    [0, 3, 2, 2],  # Transformer
    [1, 1, 3, 2],  # 小数据集分类
    [2, 1, 2, 3],  # 迁移学习
    [0, 3, 1, 0],  # 强化学习
    [1, 1, 1, 1],  # GAN生成
])

im = ax.imshow(heatmap_data, cmap='RdYlGn', aspect='auto', vmin=0, vmax=3)

# 设置刻度
ax.set_xticks(np.arange(len(techniques)))
ax.set_yticks(np.arange(len(scenarios)))
ax.set_xticklabels(techniques, fontsize=10)
ax.set_yticklabels(scenarios, fontsize=10)

# 添加数值和推荐标记
for i in range(len(scenarios)):
    for j in range(len(techniques)):
        value = heatmap_data[i, j]
        if value == 3:
            text = '⭐\n强烈\n推荐'
            color = 'white'
            weight = 'bold'
        elif value == 2:
            text = '✓\n推荐'
            color = 'black'
            weight = 'normal'
        elif value == 1:
            text = '○\n可选'
            color = 'black'
            weight = 'normal'
        else:
            text = '✗\n不推荐'
            color = 'gray'
            weight = 'normal'
        
        ax.text(j, i, text, ha='center', va='center', 
                fontsize=8, color=color, fontweight=weight)

# 添加颜色条
cbar = plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
cbar.set_label('推荐程度', fontsize=10)
cbar.set_ticks([0, 1, 2, 3])
cbar.set_ticklabels(['不推荐', '可选', '推荐', '强烈推荐'], fontsize=9)

plt.tight_layout()
plt.show()

print("\n" + "="*80)
print("🎯 快速决策指南")
print("="*80)
print("\n【第一步：看网络类型】")
print("  • CNN → 优先考虑 Batch Normalization")
print("  • RNN/Transformer → 优先考虑 Layer Normalization")
print("  • MLP → 根据深度和数据量决定")

print("\n【第二步：看Batch Size】")
print("  • Batch Size > 32 → BN效果好")
print("  • Batch Size < 8 → Layer Norm更稳定")
print("  • Batch Size = 1 → 必须用Layer Norm")

print("\n【第三步：看主要问题】")
print("  • 训练不稳定/收敛慢 → 加 BN")
print("  • 过拟合 → 加 Dropout")
print("  • 两者都有 → BN + Dropout组合")

print("\n【第四步：调整参数】")
print("  • Dropout概率：全连接层0.5，卷积层0.2")
print("  • BN位置：Dense → BN → ReLU → Dropout")
print("  • 学习率：使用BN后可以提高2-5倍")

print("\n💡 记住：实践中需要根据具体情况调整，没有万能的配置！")
print("="*80)

---

## 🧭 实用决策指南：何时使用BN/Dropout/Layer Norm？

### 决策流程图

```
开始
  │
  ├─→ 【网络类型】
  │    ├─ CNN（卷积网络）
  │    │   ├─ 卷积层：使用 Batch Normalization
  │    │   └─ 全连接层：BN + Dropout (p=0.2-0.5)
  │    │
  │    ├─ RNN/LSTM/Transformer
  │    │   └─ 使用 Layer Normalization（不依赖batch）
  │    │
  │    └─ 全连接网络（MLP）
  │        ├─ 浅层（1-2层）：可能不需要
  │        ├─ 中等（3-5层）：选BN或Dropout之一
  │        └─ 深层（>5层）：BN + Dropout组合
  │
  ├─→ 【Batch Size】
  │    ├─ 大batch (>32)：Batch Normalization效果好
  │    ├─ 小batch (<8)：Layer Normalization更稳定
  │    └─ 单样本 (=1)：必须用Layer Normalization
  │
  ├─→ 【训练数据量】
  │    ├─ 大数据集 (>10K)：BN效果好
  │    ├─ 中等数据集 (1K-10K)：BN + Dropout
  │    └─ 小数据集 (<1K)：重点用Dropout（强正则化）
  │
  └─→ 【主要问题】
       ├─ 训练不稳定/梯度消失：优先使用BN
       ├─ 过拟合：优先使用Dropout
       ├─ 收敛太慢：使用BN
       └─ 推理速度要求高：谨慎使用Dropout（训练慢）
```

### 决策矩阵表

| 场景 | 网络深度 | 数据量 | Batch Size | 推荐方案 | 理由 |
|-----|---------|--------|-----------|---------|------|
| **图像分类（CNN）** | 深 | 大 | 32-128 | **BN + Dropout** | BN稳定训练，Dropout防过拟合 |
| **目标检测** | 很深 | 大 | 小(2-8) | **Group Norm + Dropout** | 小batch下GN优于BN |
| **文本生成（RNN）** | 中-深 | 大 | 32-64 | **Layer Norm + Dropout** | LN不依赖batch，适合序列 |
| **Transformer** | 很深 | 大 | 大 | **Layer Norm + Dropout** | Transformer标配 |
| **小数据集分类** | 浅-中 | 小 | 16-32 | **只用Dropout(p=0.5)** | 重点防过拟合 |
| **迁移学习** | 深 | 小 | 16-32 | **冻结BN + Dropout** | 保留预训练的BN统计量 |
| **强化学习** | 中 | 流式 | 1-32 | **Layer Norm** | 样本非i.i.d，BN不适用 |
| **生成模型（GAN）** | 深 | 大 | 32-64 | **谨慎使用** | BN可能影响多样性 |

### 超参数选择速查表

| 参数 | 推荐范围 | 说明 |
|-----|---------|------|
| **Dropout概率** | | |
| - 全连接层 | 0.3 - 0.5 | 常用0.5 |
| - 卷积层 | 0.1 - 0.3 | 卷积层已有空间不变性，dropout少用 |
| - RNN | 0.2 - 0.5 | 只在非循环连接上dropout |
| - 输入层 | 0.1 - 0.2 | 很少用 |
| **BN动量（momentum）** | 0.9 - 0.99 | 0.9为默认值 |
| **BN epsilon** | 1e-5 - 1e-3 | 1e-5为默认，防止除零 |
| **BN位置** | 激活函数之前或之后 | Dense → BN → ReLU（推荐） |

### 组合使用建议

#### ✅ 推荐组合

1. **Dense → BN → ReLU → Dropout**（全连接层标配）
   ```python
   x = Dense(units)(x)
   x = BatchNormalization()(x)
   x = ReLU()(x)
   x = Dropout(0.5)(x)
   ```

2. **Conv → BN → ReLU → Dropout → Pool**（卷积层标配）
   ```python
   x = Conv2D(filters)(x)
   x = BatchNormalization()(x)
   x = ReLU()(x)
   x = Dropout(0.2)(x)  # 注意：卷积层dropout概率较小
   x = MaxPooling2D()(x)
   ```

3. **Transformer Block: LayerNorm + Dropout**
   ```python
   # Multi-Head Attention
   x = LayerNorm()(x + Dropout(0.1)(MultiHeadAttention(x)))
   # Feed Forward
   x = LayerNorm()(x + Dropout(0.1)(FeedForward(x)))
   ```

#### ❌ 不推荐组合

1. **BN + Layer Norm**（冗余，选一个即可）
2. **过度Dropout**（多层都用高dropout会导致欠拟合）
3. **小batch + BN**（batch<8时BN统计不稳定）

### 常见问题排查

| 问题 | 可能原因 | 解决方案 |
|-----|---------|---------|
| 训练不稳定 | 缺少归一化 | 添加BN或Layer Norm |
| 训练准确率远高于测试 | 过拟合 | 增加Dropout概率或添加更多正则化 |
| 收敛非常慢 | 学习率太小 | 使用BN后可以提高学习率2-5倍 |
| 推理时结果异常 | BN/Dropout模式错误 | 确保推理时 `training=False` |
| 小batch训练崩溃 | BN统计量不稳定 | 改用Layer Norm或Group Norm |
| 验证集loss震荡 | Dropout太强 | 降低dropout概率 |

---

---

## 3. 带BN和Dropout的完整网络

### 3.1 构建现代神经网络

In [ ]:
# 激活函数
def relu(z):
    return np.maximum(0, z)

def relu_derivative(z):
    return (z > 0).astype(float)

def softmax(z):
    if z.ndim == 1:
        z_shifted = z - np.max(z)
        exp_z = np.exp(z_shifted)
        return exp_z / np.sum(exp_z)
    else:
        z_shifted = z - np.max(z, axis=1, keepdims=True)
        exp_z = np.exp(z_shifted)
        return exp_z / np.sum(exp_z, axis=1, keepdims=True)


class ModernNeuralNetwork:
    """
    现代神经网络架构
    
    结构:
        输入 → Dense → BN → ReLU → Dropout →
        Dense → BN → ReLU → Dropout →
        Dense → Softmax
    
    参数:
        use_bn: 是否使用Batch Normalization
        use_dropout: 是否使用Dropout
        drop_prob: Dropout概率
    """
    def __init__(self, input_size=64, hidden_size=128, output_size=10,
                 use_bn=True, use_dropout=True, drop_prob=0.5):
        self.use_bn = use_bn
        self.use_dropout = use_dropout
        
        # 权重初始化（He初始化）
        self.W1 = np.random.randn(hidden_size, input_size) * np.sqrt(2.0 / input_size)
        self.b1 = np.zeros(hidden_size)
        
        self.W2 = np.random.randn(hidden_size, hidden_size) * np.sqrt(2.0 / hidden_size)
        self.b2 = np.zeros(hidden_size)
        
        self.W3 = np.random.randn(output_size, hidden_size) * np.sqrt(2.0 / hidden_size)
        self.b3 = np.zeros(output_size)
        
        # Batch Normalization层
        if use_bn:
            self.bn1 = BatchNormalization(hidden_size)
            self.bn2 = BatchNormalization(hidden_size)
        
        # Dropout层
        if use_dropout:
            self.dropout1 = Dropout(drop_prob)
            self.dropout2 = Dropout(drop_prob)
    
    def forward(self, X, training=True):
        """
        前向传播
        
        参数:
            X: 输入, shape (m, input_size)
            training: 训练模式或推理模式
        """
        # 第1层
        self.z1 = X @ self.W1.T + self.b1  # (m, hidden_size)
        
        if self.use_bn:
            self.z1 = self.bn1.forward(self.z1, training=training)
        
        self.a1 = relu(self.z1)
        
        if self.use_dropout:
            self.a1 = self.dropout1.forward(self.a1, training=training)
        
        # 第2层
        self.z2 = self.a1 @ self.W2.T + self.b2  # (m, hidden_size)
        
        if self.use_bn:
            self.z2 = self.bn2.forward(self.z2, training=training)
        
        self.a2 = relu(self.z2)
        
        if self.use_dropout:
            self.a2 = self.dropout2.forward(self.a2, training=training)
        
        # 输出层
        self.z3 = self.a2 @ self.W3.T + self.b3  # (m, output_size)
        self.a3 = softmax(self.z3)
        
        return self.a3
    
    def compute_loss(self, X, y_true, training=True):
        """交叉熵损失"""
        y_pred = self.forward(X, training=training)
        epsilon = 1e-15
        y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
        loss = -np.mean(np.sum(y_true * np.log(y_pred), axis=1))
        return loss
    
    def compute_accuracy(self, X, y_true, training=False):
        """计算准确率"""
        y_pred = self.forward(X, training=training)
        predictions = np.argmax(y_pred, axis=1)
        labels = np.argmax(y_true, axis=1)
        return np.mean(predictions == labels)


print("现代神经网络架构实现完成！")
print("\n🏗️ 架构特点：")
print("1. Batch Normalization：稳定训练")
print("2. Dropout：防止过拟合")
print("3. ReLU激活：缓解梯度消失")
print("4. 训练/推理模式自动切换")

### 3.2 MNIST数据集实战

In [ ]:
# 加载数据
digits = load_digits()
X = digits.data
y = digits.target

# 划分数据集
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 标准化
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# One-hot编码
def to_one_hot(y, n_classes=10):
    one_hot = np.zeros((len(y), n_classes))
    one_hot[np.arange(len(y)), y] = 1
    return one_hot

y_train_onehot = to_one_hot(y_train)
y_test_onehot = to_one_hot(y_test)

print(f"训练集: {X_train.shape}, 测试集: {X_test.shape}")

In [ ]:
def train_modern_network(X_train, y_train, X_test, y_test,
                          use_bn=True, use_dropout=True, 
                          n_epochs=100, batch_size=32, learning_rate=0.001):
    """
    训练现代神经网络
    """
    model = ModernNeuralNetwork(
        input_size=64, hidden_size=128, output_size=10,
        use_bn=use_bn, use_dropout=use_dropout, drop_prob=0.5
    )
    
    train_losses = []
    train_accs = []
    test_accs = []
    
    n_batches = int(np.ceil(len(X_train) / batch_size))
    
    for epoch in range(n_epochs):
        # 打乱数据
        indices = np.random.permutation(len(X_train))
        X_shuffled = X_train[indices]
        y_shuffled = y_train[indices]
        
        # Mini-batch训练（这里简化，只记录损失）
        for batch in range(n_batches):
            start = batch * batch_size
            end = min(start + batch_size, len(X_train))
            
            X_batch = X_shuffled[start:end]
            y_batch = y_shuffled[start:end]
            
            # 前向传播（训练模式）
            model.forward(X_batch, training=True)
            # 注：完整的反向传播实现较复杂，这里省略
        
        # 每个epoch结束后评估
        train_loss = model.compute_loss(X_train, y_train, training=True)
        train_acc = model.compute_accuracy(X_train, y_train, training=False)
        test_acc = model.compute_accuracy(X_test, y_test, training=False)
        
        train_losses.append(train_loss)
        train_accs.append(train_acc)
        test_accs.append(test_acc)
        
        if (epoch + 1) % 20 == 0:
            print(f"Epoch {epoch+1}/{n_epochs}, Loss: {train_loss:.4f}, "
                  f"Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}")
    
    return model, train_losses, train_accs, test_accs


# 对比不同配置
configs = [
    {'use_bn': False, 'use_dropout': False, 'name': '无BN无Dropout'},
    {'use_bn': True, 'use_dropout': False, 'name': '只有BN'},
    {'use_bn': False, 'use_dropout': True, 'name': '只有Dropout'},
    {'use_bn': True, 'use_dropout': True, 'name': 'BN+Dropout'},
]

results = {}

for config in configs:
    print(f"\n{'='*50}")
    print(f"训练配置: {config['name']}")
    print(f"{'='*50}")
    
    model, train_losses, train_accs, test_accs = train_modern_network(
        X_train, y_train_onehot, X_test, y_test_onehot,
        use_bn=config['use_bn'],
        use_dropout=config['use_dropout'],
        n_epochs=100
    )
    
    results[config['name']] = {
        'train_accs': train_accs,
        'test_accs': test_accs
    }

In [ ]:
# 可视化对比
epochs = np.arange(1, 101)

plt.figure(figsize=(14, 5))

# 测试准确率对比
plt.subplot(1, 2, 1)
for config in configs:
    name = config['name']
    plt.plot(epochs, results[name]['test_accs'], label=name, linewidth=2)
plt.xlabel('Epoch', fontsize=11)
plt.ylabel('Test Accuracy', fontsize=11)
plt.title('测试准确率对比', fontsize=12, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)

# 最终结果对比
plt.subplot(1, 2, 2)
final_test_accs = [results[cfg['name']]['test_accs'][-1] for cfg in configs]
colors = ['#95a5a6', '#3498db', '#e74c3c', '#2ecc71']
bars = plt.bar(range(len(configs)), final_test_accs, color=colors, alpha=0.8)
plt.xlabel('配置', fontsize=11)
plt.ylabel('最终测试准确率', fontsize=11)
plt.title('最终性能对比', fontsize=12, fontweight='bold')
plt.xticks(range(len(configs)), [cfg['name'] for cfg in configs], rotation=15)
plt.grid(True, alpha=0.3, axis='y')

# 在柱状图上标注数值
for i, bar in enumerate(bars):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
             f'{height:.3f}', ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.show()

print("\n📊 实验结论：")
print("1. BN+Dropout组合通常效果最好")
print("2. BN主要加速收敛")
print("3. Dropout主要防止过拟合")
print("4. 两者结合效果最佳")

---

## 4. Layer Normalization

### 4.1 BN vs LN对比

**Batch Normalization：**
- 沿着batch维度归一化
- 依赖batch size
- 适用于CNN

**Layer Normalization：**
- 沿着特征维度归一化
- 不依赖batch size
- 适用于RNN和Transformer

### 4.2 Layer Normalization实现

In [ ]:
class LayerNormalization:
    """
    层归一化（Layer Normalization）
    
    与BN的区别：
    - BN：对每个特征在batch维度上归一化
    - LN：对每个样本在特征维度上归一化
    """
    def __init__(self, n_features, epsilon=1e-5):
        self.n_features = n_features
        self.epsilon = epsilon
        
        # 可学习参数
        self.gamma = np.ones(n_features)
        self.beta = np.zeros(n_features)
    
    def forward(self, X, training=True):
        """
        前向传播
        
        参数:
            X: 输入, shape (m, n_features)
        
        注意:
            LN在训练和推理时行为相同（不依赖batch统计量）
        """
        # 计算每个样本的均值和方差（沿特征维度）
        mean = np.mean(X, axis=1, keepdims=True)  # shape: (m, 1)
        var = np.var(X, axis=1, keepdims=True)    # shape: (m, 1)
        
        # 标准化
        X_normalized = (X - mean) / np.sqrt(var + self.epsilon)
        
        # 缩放和平移
        out = self.gamma * X_normalized + self.beta
        
        # 缓存（用于反向传播）
        self.cache = {
            'X': X,
            'X_normalized': X_normalized,
            'mean': mean,
            'var': var
        }
        
        return out


# 对比BN和LN
np.random.seed(42)
X_test = np.random.randn(4, 6)  # 4个样本，6个特征

bn = BatchNormalization(n_features=6)
ln = LayerNormalization(n_features=6)

X_bn = bn.forward(X_test, training=True)
X_ln = ln.forward(X_test, training=True)

print("原始数据：")
print(X_test)
print("\nBatch Normalization（每列归一化）：")
print(X_bn)
print(f"每列均值: {np.mean(X_bn, axis=0)}")
print(f"每列方差: {np.var(X_bn, axis=0)}")

print("\nLayer Normalization（每行归一化）：")
print(X_ln)
print(f"每行均值: {np.mean(X_ln, axis=1)}")
print(f"每行方差: {np.var(X_ln, axis=1)}")

---

## 5. 本章总结

### 技术对比

| 技术 | 作用 | 优点 | 缺点 | 适用场景 |
|-----|------|------|------|----------|
| **Batch Normalization** | 稳定训练 | 加速收敛，允许更大学习率 | 依赖batch size | CNN |
| **Layer Normalization** | 稳定训练 | 不依赖batch | 效果略弱于BN | RNN, Transformer |
| **Dropout** | 防止过拟合 | 强正则化效果 | 训练时间长 | 全连接层 |

### 训练 vs 推理模式

**Batch Normalization：**
- 训练时：使用batch统计量，更新running统计量
- 推理时：使用固定的running统计量

**Dropout：**
- 训练时：随机丢弃+缩放（Inverted Dropout）
- 推理时：不丢弃，直接使用

**Layer Normalization：**
- 训练和推理时行为相同

### 实践建议

```python
# CNN架构
Conv → BN → ReLU → Dropout → Pool

# 全连接网络
Dense → BN → ReLU → Dropout

# RNN/Transformer
Dense → Layer Norm → Activation
```

### 超参数选择

- **Dropout概率**：0.2-0.5（全连接层），0.1-0.3（卷积层）
- **BN动量**：0.9-0.99
- **BN epsilon**：1e-5（默认）

---

## 6. 练习题

### 练习1：实现完整的反向传播
为`ModernNeuralNetwork`类实现完整的反向传播，包括：
- BN层的梯度计算
- Dropout层的梯度计算
- 端到端训练

### 练习2：调整Dropout位置
实验不同的Dropout放置策略：
1. 只在最后一层
2. 在所有隐藏层
3. 只在全连接层，不在卷积层

对比效果。

### 练习3：BN的可视化
可视化训练过程中：
1. 每层激活值的分布变化
2. 有BN vs 无BN的激活分布对比
3. 梯度流动的差异

### 练习4：实现Group Normalization
Group Normalization是BN和LN的折中：
- 将特征分成若干组
- 在每组内进行归一化
- 适用于小batch size场景

---

## 下一章预告

在下一章《CNN基础》中，我们将学习：
- 卷积层的原理和实现
- 池化层
- 经典CNN架构
- 图像分类实战

---

**🎓 恭喜完成第7章！你已经掌握了现代深度学习的核心技术！**